<a href="https://colab.research.google.com/github/Amisha1019/Emotion-Detection-Internship-/blob/main/Task_1_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
zyan1999_student_engagement_and_emotion_recognition_dataset_path = kagglehub.dataset_download('zyan1999/student-engagement-and-emotion-recognition-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/student-engagement-and-emotion-recognition-dataset/Student_engagement_dataset.csv
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/4058_aug_4.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/8285_original.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/6642_aug_3.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/1101_aug_3.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/951_aug_2.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/6603_aug_1.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/2250_original.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/8828_original.jpg
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy/29

In [ ]:
!pip install --upgrade protobuf==3.20.*
!pip install tensorflow==2.12.0 --quiet


In [ ]:
# train_emotion_model.py
# Kaggle-ready script to train the simple CNN emotion model on FER2013.
# Place fer2013.csv in /kaggle/input/fer2013/fer2013.csv (Kaggle FER2013 dataset)
# Run on Kaggle:
#   python train_emotion_model.py --fer_csv /kaggle/input/fer2013/fer2013.csv --epochs 30 --batch_size 64

import os
import argparse
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
def build_simple_cnn(input_shape=(48,48,1), num_classes=7):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
def load_fer2013(CSV_PATH):
    # FER2013 CSV columns: 'emotion','pixels','Usage'
    df = pd.read_csv("/kaggle/input/student-engagement-and-emotion-recognition-dataset/Student_engagement_dataset.csv")
    def parse_pixels(p):
        arr = np.fromstring(p, sep=' ', dtype=np.uint8)
        return arr.reshape(48,48)
    X = np.stack(df['pixels'].apply(parse_pixels).values)
    X = X.astype('float32') / 255.0
    X = X.reshape(-1,48,48,1)
    y = to_categorical(df['emotion'].values, num_classes=7)
    usage = df['Usage'].values
    X_train = X[usage == 'Training']
    y_train = y[usage == 'Training']
    X_val = X[usage == 'PublicTest']
    y_val = y[usage == 'PublicTest']
    X_test = X[usage == 'PrivateTest']
    y_test = y[usage == 'PrivateTest']
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


In [ ]:
def main(args):
    if not os.path.exists(args.fer_csv):
        raise FileNotFoundError(f"FER CSV not found at {args.fer_csv}. Upload fer2013.csv to Kaggle and provide the path.")
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_fer2013(args.fer_csv)
    print("Shapes: ", X_train.shape, y_train.shape, X_val.shape, y_val.shape)

    model = build_simple_cnn()
    model.compile(optimizer=Adam(learning_rate=args.lr), loss='categorical_crossentropy', metrics=['accuracy'])

    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    )
    datagen.fit(X_train)

    callbacks = [
        ModelCheckpoint('/kaggle/working/emotion_model_trained.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
        EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    ]

    model.fit(datagen.flow(X_train, y_train, batch_size=args.batch_size),
              epochs=args.epochs,
              validation_data=(X_val, y_val),
              callbacks=callbacks,
              steps_per_epoch=max(1, len(X_train)//args.batch_size))

    loss, acc = model.evaluate(X_test, y_test, verbose=2)
    print(f"Test accuracy: {acc:.4f}")
    print("Saved trained model to /kaggle/working/emotion_model_trained.h5")


In [ ]:
import os
for root, dirs, files in os.walk("/kaggle/input/student-engagement-and-emotion-recognition-dataset"):
    print(root, files)


/kaggle/input/student-engagement-and-emotion-recognition-dataset ['Student_engagement_dataset.csv']
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive []
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR []
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val []
/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/val/joy ['4058_aug_4.jpg', '8285_original.jpg', '6642_aug_3.jpg', '1101_aug_3.jpg', '951_aug_2.jpg', '6603_aug_1.jpg', '2250_original.jpg', '8828_original.jpg', '29_aug_4.jpg', '190_aug_4.jpg', '8193_aug_3.jpg', '5487_aug_2.jpg', '1273_original.jpg', '1820_aug_2.jpg', '6636_original.jpg', '6690_aug_1.jpg', '8156_aug_4.jpg', '7505_original.jpg', '1724_aug_3.jpg', '1723_aug_4.jpg', '247_original.jpg', '1888_aug_2.jpg', '3934_aug_3.jpg', '1187_original.jpg', '8830_aug_4.jpg', '8796_aug_2.jpg', '6808_original.jpg', '6715_aug_2.jpg', '7340_aug_4.jpg', '266_original.jpg', '7050_a

In [ ]:
class Args:
    fer_csv = "/kaggle/input/student-engagement-and-emotion-recognition-dataset"
    epochs = 30
    batch_size = 64
    lr = 1e-3


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
import os

for root, dirs, files in os.walk("/kaggle"):
    if "emotion_model.h5" in files:
        print("FOUND:", root, files)


In [ ]:
model = load_model("emotion_model.h5")

# Emotion labels (modify if your dataset has different classes)
emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']



FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'emotion_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# -----------------------------
# 1. Load Emotion Model
# -----------------------------
model_path = "/kaggle/input/<your-model-folder>/emotion_model.h5"
model = load_model(model_path)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

# -----------------------------
# 2. Load Image
# -----------------------------
img_path = "/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/test/joy/100_aug_4.jpg"

img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
face = cv2.resize(gray, (48,48))
face = face.reshape(1,48,48,1) / 255.0

# -----------------------------
# 3. Predict Emotion
# -----------------------------
prediction = model.predict(face)
emotion_index = np.argmax(prediction)
emotion_text = emotion_labels[emotion_index]

print("Predicted Emotion:", emotion_text)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/kaggle/input/<your-model-folder>/emotion_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import os

img_path = "/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/test"

print("Exists:", os.path.exists(img_path))


Exists: True


In [ ]:
img_path = "/kaggle/input/student-engagement-and-emotion-recognition-dataset/archive/FESR/test/upset/1003_aug_3.jpg"

img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
face = cv2.resize(gray, (48,48))
face = face.reshape(1,48,48,1) / 255.0

# -----------------------------
# 3. Predict Emotion
# -----------------------------
prediction = model.predict(face)
emotion_index = np.argmax(prediction)
emotion_text = emotion_labels[emotion_index]

# -----------------------------
# 4. Draw prediction on image
# -----------------------------
output_image = img.copy()
cv2.putText(output_image, emotion_text, (20,40),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

# -----------------------------
# 5. Show final image
# -----------------------------
plt.figure(figsize=(6,6))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title(f"Predicted Emotion: {emotion_text}")
plt.show()

# -----------------------------
# 6. Save final result
# -----------------------------
cv2.imwrite("emotion_prediction_output.jpg", output_image)
print("Saved: emotion_prediction_output.jpg")


NameError: name 'model' is not defined